### Read Data

In [ ]:
# Load data from a CSV
file_location = "/FileStore/tables/game_skater_stats.csv"
df = spark.read.format("CSV").option("inferSchema", True).option("header", True).load(file_location)
display(df.take(5))

### Write Data

In [ ]:
# Save as CSV and parquet

# DBFS
df.write.save('/FileStore/parquet/game__stats', format='parquet')

# S3
#df.write.parquet("s3a://my_bucket/game_skater_stats", mode="overwrite")

# DBFS
df.write.save('/FileStore/parquet/game__stats.csv', format='csv')

# S3
#df.coalesce(1).write.format("com.databricks.spark.csv")
#   .option("header", "true").save("s3a://my_bucket/game_skater_stats.csv")

### Transform Data

In [ ]:

df.createOrReplaceTempView("stats")

display(spark.sql("""
  select player_id, sum(1) as games, sum(goals) as goals
  from stats
  group by 1
  order by 3 desc
  limit 5
"""))

In [ ]:

# player names
file_location = "/FileStore/tables/player_info.csv"
names = spark.read.format("CSV").option("inferSchema", True).option("header", True).load(file_location)
#display(names)

In [ ]:
df.createOrReplaceTempView("stats")

top_players = spark.sql("""
select player_id, sum(1) as games, sum(goals) as goals
from stats
group by 1
order by 3 desc
limit 5
""")

top_players.createOrReplaceTempView("top_players")
names.createOrReplaceTempView("names")

display(spark.sql("""
select p.player_id, goals, firstName, lastName
from top_players p
join names n
  on p.player_id = n.player_id
order by 2 desc  
"""))

In [ ]:

display(spark.sql("""
select cast(substring(game_id, 1, 4) || '-' 
  || substring(game_id, 5, 2) || '-01' as Date) as month
  , sum(goals)/count(distinct game_id) as goals_per_goal
from stats
group by 1
order by 1
"""))

In [ ]:
:
display(spark.sql("""
select cast(goals/shots * 50 as int)/50.0 as Goals_per_shot, sum(1) as Players 
from (
  select player_id, sum(shots) as shots, sum(goals) as goals
  from stats
  group by 1
  having goals >= 5
)  
group by 1
order by 1
"""))